In [17]:
# import packages
import ee
import geemap

import pandas as pd

In [18]:
# initialize the EE api
ee.Initialize()

In [ ]:
# define EE datasets
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")
spawn = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")
countries = ee.FeatureCollection("UN/Geodata/BNDA_simplified/current")
y2y_ecoregions = ee.FeatureCollection("projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_soilgrids = ee.Image("projects/soilgrids-isric/ocs_mean")
soc_sothe = ee.Image("projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_socs = ee.Image("projects/ee-bermane/assets/SOCS_0_1m_mg_ha_year_2010")

In [20]:
# Define function to calculate zonal statistics
# specify image to use and reducer type (sum(), mean())
def calc_zonal_stats(feature, image_list, reducer_list):

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats = image.reduceRegion(
            reducer=reducer,
            geometry=feature.geometry(),
            crs=image.projection(),
            crsTransform=image.projection().getInfo().get('transform'),
            maxPixels=1e15
        )

        # add zonal stats to feature properties
        feature = feature.set(stats)
    return feature

# Define function to calculate feature area in km2
def calc_feature_area(feature):
    area_km2 = feature.geometry().area().divide(1e6)
    return feature.set({'area_km2_gee': area_km2})

# Define function to convert FeatureCollection to DataFrame
def fc_to_df(fc):
    """Extracts properties from FeatureCollection and converts to Pandas DataFrame."""
    features = fc.toList(fc.size()).getInfo()  # Convert to list
    data = [feature["properties"] for feature in features]  # Extract properties
    return pd.DataFrame(data)

In [21]:
# calc 2020 ESA CCI biomass
# grab the 2020 AGB images
agb_2020 = agb.filter(ee.Filter.stringContains("system:index", "2020")).first()

# add BGB to 2020 image using global rsr map
bio_2020 = agb_2020.addBands(agb_2020.select(['AGB']).multiply(rsr).rename('BGB')).select(['AGB', 'BGB'])

# add total carbon to 2020 image
bio_2020 = bio_2020.addBands(bio_2020.select(['AGB']).add(bio_2020.select(['BGB'])).rename('total_c'))

# rename bands
bio_2020 = bio_2020.rename(['agb_dens', 'bgb_dens', 'total_dens'])

# multiply values by 0.5 to get carbon density
bio_2020 = bio_2020.multiply(0.5).updateMask(lc.neq(18).And(lc.neq(19)))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# calculate total biomass c per pixel and rename bands
bio_stock_2020 = bio_2020.multiply(pixel_area_ha).rename(['agb_c', 'bgb_c', 'total_c'])

In [22]:
# calc open land map soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename('soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_olm_stock = soc_olm.multiply(pixel_area_ha).rename('total_olm_soc')

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [23]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename('soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_blend_dens').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('total_blend_soc')

In [24]:
# calc socs soc
# mask water/snow/ice
soc_socs = soc_socs.rename('soc_socs_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# multiply by pixel area to get total carbon per pixel
soc_socs_stock = soc_socs.multiply(pixel_area_ha).rename('total_soc_socs')

In [25]:
# check scale of layers
print('Biomass Carbon Density Scale: ', bio_2020.projection().nominalScale().getInfo())
print('Biomass Carbon Total Scale: ', bio_stock_2020.projection().nominalScale().getInfo())
print('SOC Density Sothe Scale: ', soc_sothe.projection().nominalScale().getInfo())
print('SOC Density Blend Scale: ', soc_blend.projection().nominalScale().getInfo())
print('SOC Total Blend Scale: ', soc_blend_stock.projection().nominalScale().getInfo())
print('SOC Density OLM Scale: ', soc_olm.projection().nominalScale().getInfo())

Biomass Carbon Density Scale:  98.95065848290943
Biomass Carbon Total Scale:  98.95065848290943
SOC Density Sothe Scale:  250.00021298953834
SOC Density Blend Scale:  250.00021298953834
SOC Total Blend Scale:  250.00021298953834
SOC Density OLM Scale:  231.91556871282307


## Compute zonal statistics and convert results to dataframe

In [29]:
# define image and reducer list for zonal stats
img_list = [bio_stock_2020, bio_2020, soc_blend_stock, soc_blend, soc_socs_stock, soc_socs]
redu_list = [ee.Reducer.sum(), ee.Reducer.mean(), ee.Reducer.sum(), ee.Reducer.mean(), ee.Reducer.sum(), ee.Reducer.mean()]

# Compute zonal statistics across Y2Y region
y2y_carbon = y2y.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area)

# Compute zonal statistics across ecoregions
eco_carbon = y2y_ecoregions.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area)

# Compute zonal statistics across biomes
biome_carbon = y2y_biomes.map(lambda feature: calc_zonal_stats(feature, img_list, redu_list)).map(calc_feature_area)

# Compute zonal statistics across Canada
canada = countries.filter(ee.Filter.eq('lbl_en', 'CANADA'))
img_list_can = [bio_2020, soc_blend, soc_socs]
redu_list_can = [ee.Reducer.mean(), ee.Reducer.mean(), ee.Reducer.mean()]
canada_carbon = canada.map(lambda feature: calc_zonal_stats(feature, img_list_can, redu_list_can)).map(calc_feature_area)

In [32]:
# export results to google drive
ee.batch.Export.table.toDrive(
    collection=y2y_carbon,
    description="y2y_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

ee.batch.Export.table.toDrive(
    collection=eco_carbon,
    description="y2y_ecoregions_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

ee.batch.Export.table.toDrive(
    collection=biome_carbon,
    description="y2y_biomes_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

ee.batch.Export.table.toDrive(
    collection=canada_carbon,
    description="canada_carbon_stock",
    folder="carbon_stock_outputs",
    fileFormat="CSV"
).start()

## Add layers to map

In [27]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = {
    1: "#033e00",  # Temperate or sub-polar needleleaf forest
    2: "#939b71",  # Sub-polar taiga needleleaf forest
    3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    6: "#6b7d2c",  # Mixed forest
    7: "#b29d29",  # Tropical or sub-tropical shrubland
    8: "#b48833",  # Temperate or sub-polar shrubland
    9: "#e9da5d",  # Tropical or sub-tropical grassland
    10: "#e0cd88",  # Temperate or sub-polar grassland
    11: "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    12: "#bad292",  # Sub-polar or polar grassland-lichen-moss
    13: "#3f8970",  # Sub-polar or polar barren-lichen-moss
    14: "#6ca289",  # Wetland
    15: "#e6ad6a",  # Cropland
    16: "#a9abae",  # Barren land
    17: "#db2126",  # Urban and built-up
    18: "#4c73a1",  # Water
    19: "#fff7fe",  # Snow and ice
}

In [28]:
# create a map
m = geemap.Map()

# add layers
m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")

# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…